<a href="https://colab.research.google.com/github/Varshinigarlapati963/Code_Tech_03/blob/main/CodeTech_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nltk


In [3]:
# Import required libraries
import nltk
import numpy as np
import json
import random

from nltk.stem import PorterStemmer
from sklearn.naive_bayes import MultinomialNB


In [4]:
# Save intents.json
import json

intents_data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey"],
            "responses": ["Hello!", "Hi there!", "Hey! How can I help you?"]
        },
        {
            "tag": "name",
            "patterns": ["What is your name?", "Who are you?", "Tell me your name"],
            "responses": ["I'm CodTech Bot!", "I am your AI assistant."]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "Goodbye", "See you later"],
            "responses": ["Goodbye!", "Have a great day!", "See you again!"]
        }
    ]
}

with open("intents.json", "w") as f:
    json.dump(intents_data, f, indent=4)


In [5]:
# Custom tokenizer to avoid punkt errors
def simple_tokenize(text):
    return text.lower().replace("?", "").replace("!", "").replace(".", "").replace(",", "").split()

# Initialize
stemmer = PorterStemmer()
all_words = []
tags = []
xy = []

# Load intents
with open("intents.json") as f:
    intents = json.load(f)

# Tokenize and prepare word/tag pairs
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = simple_tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

# Stem and clean
all_words = [stemmer.stem(w) for w in all_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))


In [6]:
def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stemmer.stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [7]:
model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

In [10]:
def predict_tag(sentence):
    tokens = simple_tokenize(sentence)
    bow = bag_of_words(tokens, all_words).reshape(1, -1)
    result = model.predict(bow)[0]
    return tags[result]

def get_response(tag):
    for intent in intents['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])

def chat():
    print("CodTechBot: Hello! Type 'quit' to exit.")
    while True:
        sentence = input("You: ")
        if sentence.lower() == "quit":
            print("CodTechBot: Goodbye!")
            break
        tag = predict_tag(sentence)
        response = get_response(tag)
        print("Urs ChatBot:", response)


In [11]:
chat()


CodTechBot: Hello! Type 'quit' to exit.
You: hi
Urs ChatBot: Hello!
You: whats ur name'
Urs ChatBot: I'm CodTech Bot!
You: what are u doing
Urs ChatBot: I'm CodTech Bot!
You: see you later
Urs ChatBot: See you again!
You: bye
Urs ChatBot: See you again!
You: good bye
Urs ChatBot: Have a great day!
You: quit
CodTechBot: Goodbye!


In [32]:
# !pip install nltk textblob scikit-learn

import nltk
import json
import random
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB
import os

# Ensure NLTK data is available
nltk.download('punkt')
nltk.download('wordnet')

# Add additional search paths (helps in Colab or Docker)
nltk.data.path.extend([
    '/usr/share/nltk_data', '/usr/local/share/nltk_data',
    '/usr/lib/nltk_data', '/usr/local/lib/nltk_data',
    '/root/nltk_data'
])

# ========== Define Intents ==========
intents = {
    "intents": [
        {"tag": "greeting", "patterns": ["Hi", "Hello", "Hey", "Yo", "Hlo", "hiya"],
         "responses": ["Hi there! 👋", "Hello! 😊", "Hey! How can I help you?"]},
        {"tag": "goodbye", "patterns": ["Bye", "Goodbye", "See you", "Catch you later"],
         "responses": ["Goodbye! 👋", "See you soon!", "Take care! 😊"]},
        {"tag": "thanks", "patterns": ["Thanks", "Thank you", "Appreciate it", "Thx"],
         "responses": ["You're welcome! 😊", "Glad to help!", "Anytime! 🙌"]},
        {"tag": "name", "patterns": ["What's your name?", "Your name?", "Who are you?"],
         "responses": ["I'm CodTech Bot 🤖", "You can call me CodTech Bot."]},
        {"tag": "capabilities", "patterns": ["What can you do?", "How can you help?", "Your features?"],
         "responses": ["I can chat with you, answer questions, and assist with tasks."]},
        {"tag": "creator", "patterns": ["Who created you?", "Who made you?", "Your developer?"],
         "responses": ["A talented developer like you built me!", "I was made with Python & care! 💻"]},
        {"tag": "mood", "patterns": ["How are you?", "How's it going?", "Are you okay?"],
         "responses": ["I'm great! Ready to help you. 😊", "Feeling code-tastic!"]},
        {"tag": "help", "patterns": ["I need help", "Can you help me?", "Assist me please"],
         "responses": ["Sure, tell me how I can help!", "I'm here for you. What do you need?"]},
        {"tag": "unknown", "patterns": [],
         "responses": ["I'm not sure what you mean. 🤔", "Can you rephrase that?"]}
    ]
}

# ========== Save Intents ==========
with open("intents.json", "w") as f:
    json.dump(intents, f, indent=4)

# ========== Preprocessing ==========
lemmatizer = WordNetLemmatizer()

def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text.lower())
    return [lemmatizer.lemmatize(w) for w in tokens if w.isalpha()]

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        tokens = tokenize_and_lemmatize(pattern)
        all_words.extend(tokens)
        xy.append((tokens, tag))

# Remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

def bag_of_words(tokenized_sentence, all_words):
    sentence_words = [lemmatizer.lemmatize(w.lower()) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, word in enumerate(all_words):
        if word in sentence_words:
            bag[idx] = 1
    return bag

# ========== Training ==========
X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    y_train.append(tags.index(tag))

X_train = np.array(X_train)
y_train = np.array(y_train)

model = MultinomialNB()
model.fit(X_train, y_train)

# ========== Inference ==========
def predict_tag(sentence):
    tokens = tokenize_and_lemmatize(sentence)
    bow = bag_of_words(tokens, all_words).reshape(1, -1)
    probs = model.predict_proba(bow)[0]
    max_prob = np.max(probs)
    predicted_index = np.argmax(probs)

    if max_prob > 0.5:
        return tags[predicted_index]
    else:
        return "unknown"

def get_response(tag):
    for intent in intents['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])

# ========== Chat Console ==========
def chat():
    print("CodTechBot: Hello! Type 'quit' to exit.")
    while True:
        sentence = input("You: ")
        if sentence.lower() == "quit":
            print("CodTechBot: Goodbye! 👋")
            break
        tag = predict_tag(sentence)
        response = get_response(tag)
        print("CodTechBot:", response)

# ========== Start Chat ==========
chat()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


CodTechBot: Hello! Type 'quit' to exit.
You: hi
CodTechBot: Can you rephrase that?
You: Hi
CodTechBot: I'm not sure what you mean. 🤔
You: Hello
CodTechBot: Can you rephrase that?
You: what is ur name
CodTechBot: Can you rephrase that?
You: quit
CodTechBot: Goodbye! 👋


In [36]:
# Required Libraries
import nltk
import numpy as np
import json
import random
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB

# Download NLTK Data
nltk.download('punkt')
nltk.download('wordnet')

# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents
intents_data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey", "Good morning", "Good evening", "Hi there", "What's up", "Yo", "Howdy"],
            "responses": ["Hello! 😊", "Hi there! 👋", "Hey! How can I help you?"]
        },
        {
            "tag": "name",
            "patterns": ["What is your name?", "Who are you?", "Tell me your name", "What's your name", "Your name?"],
            "responses": ["I'm CodTech Bot! 🤖", "I am your AI assistant."]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "Goodbye", "See you later", "See ya", "Catch you later", "Take care"],
            "responses": ["Goodbye! 👋", "Have a great day! 🌞", "See you again!"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "Appreciate it", "Thx", "Thanks a lot", "Much appreciated"],
            "responses": ["You're welcome! 😊", "Anytime!", "Glad to help! 🙌"]
        },
        {
            "tag": "capabilities",
            "patterns": ["What can you do?", "What are your features?", "What do you know?", "How can you help me?"],
            "responses": [
                "I can answer questions, chat with you, and help you learn things! 📚",
                "I'm here to assist with small talk and basic questions.",
                "I can provide information, respond to greetings, and more!"
            ]
        },
        {
            "tag": "creator",
            "patterns": ["Who created you?", "Who made you?", "Your developer?", "Who built you?"],
            "responses": ["I was created by a developer just like you!", "An awesome human programmed me. 🤖"]
        },
        {
            "tag": "mood",
            "patterns": ["How are you?", "How's it going?", "Are you okay?", "You good?", "How do you feel?"],
            "responses": ["I'm doing great, thanks for asking! 😊", "Feeling code-tastic!", "Always ready to chat!"]
        }
    ]
}

# Save to intents.json (optional)
with open("intents.json", "w") as f:
    json.dump(intents_data, f, indent=4)

# Tokenization and Preprocessing
def clean_text(text):
    text = re.sub(r"[^\w\s]", "", text.lower())
    return word_tokenize(text)

all_words = []
tags = []
xy = []

for intent in intents_data['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        tokens = clean_text(pattern)
        tokens = [lemmatizer.lemmatize(w) for w in tokens]
        all_words.extend(tokens)
        xy.append((tokens, tag))

# Remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

# Bag of Words
def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [lemmatizer.lemmatize(w.lower()) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, word in enumerate(all_words):
        if word in tokenized_sentence:
            bag[idx] = 1.0
    return bag

# Training data
X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bow = bag_of_words(pattern_sentence, all_words)
    X_train.append(bow)
    y_train.append(tags.index(tag))

X_train = np.array(X_train)
y_train = np.array(y_train)

# Train the model
model = MultinomialNB()
model.fit(X_train, y_train)

# Prediction with confidence threshold
def predict_tag(sentence, threshold=0.3):
    sentence_tokens = clean_text(sentence)
    bow = bag_of_words(sentence_tokens, all_words).reshape(1, -1)
    probs = model.predict_proba(bow)[0]
    best_index = np.argmax(probs)
    if probs[best_index] >= threshold:
        return tags[best_index]
    return "unknown"

def get_response(tag):
    for intent in intents_data['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    return random.choice([
        "Hmm... I'm not sure what you mean. 🤔",
        "I'm sorry, I didn't understand that. 😕",
        "Could you rephrase it?"
    ])

# Console Chat
def chat():
    print("CodTechBot: Hello! Type 'quit' to exit.")
    while True:
        msg = input("You: ")
        if msg.lower() == "quit":
            print("CodTechBot: Goodbye! 👋")
            break
        tag = predict_tag(msg)
        response = get_response(tag)
        print("Urs ChatBot:", response)

# Run
chat()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


CodTechBot: Hello! Type 'quit' to exit.
You: hi
Urs ChatBot: Hello! 😊
You: how are you
Urs ChatBot: I'm doing great, thanks for asking! 😊
You: what can you do
Urs ChatBot: I can provide information, respond to greetings, and more!
You: ur name?
Urs ChatBot: I'm CodTech Bot! 🤖
You: quit
CodTechBot: Goodbye! 👋
